Use binary search for find the robust cube from a single sample point in the region 18:1

In [4]:
import json
from goodDesignHelpers import *
import sys

In [5]:
def generateAtomicIndex(parameter):
    parameter[:2] = sorted(parameter[:2])[::-1]
    theta1, theta2, mu,beta, pi, d = parameter
    factor = (pi*pi + 2*pi*d)*mu
    #factor = 1

    dl2 = beta*(pi+d) + (1+beta)*d
    du2 = beta*d + (1+beta)*(pi+d)
    dl1 = beta*d + (1+beta)*(pi+d)
    tl2 = (beta*mu*(pi+d)+(2+2*beta+mu+beta*mu)*d)/(1+mu)
    tu1 = (beta*mu*d + (2+2*beta+mu+beta*mu)*(pi+d))/(1+mu)
    ru2 = ((mu+beta*mu)*(pi+d)+(2+2*beta+mu+beta*mu)*d)/(1+mu)
    rl1 = ((mu+beta*mu)*(pi+d)+(2+2*beta+mu+beta*mu)*d)/(1+mu)
    ru1 = ((mu+beta*mu)*d+(2+2*beta+mu+mu*beta)*(pi+d))/(1+mu)
    sl12 = beta*(2*d+pi)
    su22 = (1+beta)*(2*d+pi)
    sl21 = (1+beta)*(2*d+pi)
    su31 = (1+beta)*(2*d+pi)

    scaledList1 = [dl1,tu1,rl1,ru1,sl21,su31]
    scaledList2 = [dl2,du2,tl2,ru2,sl12,su22]

    theta1List = [x/factor for x in scaledList1]
    theta2List = [x/factor for x in scaledList2] 

    pre1 = []
    post1 = []
    for i in range(len(theta1List)):
        curr = theta1List[i]
        if curr < theta1:
            pre1.append(i)
        elif curr > theta1:
            post1.append(i)
        else:
            # should return False
            pass

    pre2 = []
    post2 = []
    for i in range(len(theta2List)):
        curr = theta2List[i]
        if curr < theta2:
            pre2.append(i)
        elif curr > theta2:
            post2.append(i)
        else:
            # should return False
            pass
    index = [pre1,post1,pre2,post2]
    index = [tuple(x) for x in index]
    return tuple(index)

def isGoodDesign(portrait):
    if len(portrait) > 0:
        if (0 not in portrait) and (4 not in portrait) and (8 not in portrait):
            return True
    return False

def getPortrait(parameters,H1,H2):
    theta1, theta2, mu, beta, pi, d = parameters
    numericalPF = partial(findFP,params = parameters,H1=H1,H2=H2)
    upperBound = (beta+1)/(mu*pi) + (1+beta)/(mu*(pi+d))
    samples = regionSample(theta1,theta2,upperBound)
    portrait = dict()
    #equilibrium = dict()
    for i in range(len(samples)):
        for j in range(len(samples)):
            p1, p2 = samples[i], samples[j]
            x = fsolve(numericalPF,[p1,p2]) 

            if not sum(map(lambda k: abs(k), numericalPF(x))) < 1e-6:
                continue 
            #print(x,numericalPF(x))
            if isInRegion(x,parameters) and isAttractorOverall(x,parameters,H1,H2):
                generatePortraits(x,parameters,portrait)
    
    return portrait

    
    
def bsMethod(start, direction, history, H1, H2):
    """as the region is simply connected """
    t0,t1 =0, 1
    #end = [(start[i]+direction[i])/2 for i in range(len(left))]
    #endPortrait = getPortrait(end, H1, H2)
    #isGoodEnd = isGoodDesign(endPortrait)
    #history.append((end, isGoodEnd))
    
    updateDistance = t1 - t0
    
    while updateDistance > 1e-6:
        
        mt = (t0+t1)/2
        mPoint = [start[i]+direction[i]*mt for i in range(len(direction))]
        mPortrait = getPortrait(mPoint,H1,H2)
        isGoodM = isGoodDesign(mPortrait)
        
        if isGoodM:
            t0 = mt
        else:
            t1 = mt
            
        history.append((t0,t1))
        
        updateDistance = t1 - t0
            
            
def getUpperBound(start, direction, history, H1, H2):
    t = 1
    while True:
        ePoint = [start[i]+direction[i]*t for i in range(len(direction))]
        ePortrait = getPortrait(ePoint,H1,H2)
        isGoodE = isGoodDesign(ePortrait)
        history.append((t,isGoodE))
        
        if isGoodE:
            t*=2
            
        if not isGoodE or t > 1e6:
            return 
        

        
def getRobustCube(point, H1, H2):
    pass


def getDirectionIntervale():
    pass

 (\theta_1, \theta_2, \mu, \beta, \pi, D) = (1.8, 0.67, 0.275, 0.45, 2.95, 0.8)

In [6]:
parameters = [1.8, 0.67, 0.275, 0.45, 2.95, 0.8]

In [7]:
v=20
portraitList = []
v1 = v2 = v
H1= partial(Hill,v = v1)
H2= partial(Hill,v = v2)


count = 1

ret = []
subSample = []

count = 0

    # check the portrait of each sample for 
#for key in keys:
    #parameters = parameterList[key]
    #print(k)
theta1, theta2, mu, beta, pi, d = parameters
numericalPF = partial(findFP,params = parameters,H1=H1,H2=H2)
upperBound = (beta+1)/(mu*pi) + (1+beta)/(mu*(pi+d))
samples = regionSample(theta1,theta2,upperBound)
portrait = dict()
#equilibrium = dict()
for i in range(len(samples)):
    for j in range(len(samples)):
        p1, p2 = samples[i], samples[j]
        x = fsolve(numericalPF,[p1,p2]) 

        if not sum(map(lambda k: abs(k), numericalPF(x))) < 1e-6:
            continue 
        #print(x,numericalPF(x))
        if isInRegion(x,parameters) and isAttractorOverall(x,parameters,H1,H2):
            generatePortraits(x,parameters,portrait)

# is asymmetric
if isGoodDesign(portrait):
    print(portrait)
    ret.append(list(parameters))



/Users/lunzhang/anaconda3/envs/tensorflow_env/lib/python3.6/site-packages/scipy/optimize/minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/Users/lunzhang/anaconda3/envs/tensorflow_env/lib/python3.6/site-packages/scipy/optimize/minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


{7: {(0.877543, 2.695988)}, 5: {(2.695988, 0.877543)}}


In [8]:
def getPortrait(parameters, H1, H2):
    theta1, theta2, mu, beta, pi, d = parameters
    numericalPF = partial(findFP,params = parameters,H1=H1,H2=H2)
    upperBound = (beta+1)/(mu*pi) + (1+beta)/(mu*(pi+d))
    samples = regionSample(theta1,theta2,upperBound)
    portrait = dict()
    #equilibrium = dict()
    for i in range(len(samples)):
        for j in range(len(samples)):
            p1, p2 = samples[i], samples[j]
            x = fsolve(numericalPF,[p1,p2]) 

            if not sum(map(lambda k: abs(k), numericalPF(x))) < 1e-6:
                continue 
            #print(x,numericalPF(x))
            if isInRegion(x,parameters) and isAttractorOverall(x,parameters,H1,H2):
                generatePortraits(x,parameters,portrait)

    # is asymmetric
    return portrait


In [9]:
v=20
portraitList = []
v1 = v2 = v
H1= partial(Hill,v = v1)
H2= partial(Hill,v = v2)
parameters = [1.8, 0.67, 0.275, 0.45, 2.95, 0.8]
getPortrait(parameters,H1, H2)

/Users/lunzhang/anaconda3/envs/tensorflow_env/lib/python3.6/site-packages/scipy/optimize/minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/Users/lunzhang/anaconda3/envs/tensorflow_env/lib/python3.6/site-packages/scipy/optimize/minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


{7: {(0.877543, 2.695988)}, 5: {(2.695988, 0.877543)}}

In [39]:
count

54

In [ ]:
count = 1
proportion = 0.2
position = 5
while True:
    params = parameters.copy()
    params[position] += count*proportion*parameters[position]
    portrait = getPortrait(params,H1, H2)
    print(portrait)
    count+=1

In [ ]:
count = 1
proportion = 0.02
position = 5
while True:
    params = parameters.copy()
    params[position] -= count*proportion*parameters[position]
    if params[position] <=0:
        break
    print(getPortrait(params,H1, H2),count)
    count+=1

In [33]:
v=20
portraitList = []
v1 = v2 = v
H1= partial(Hill,v = v1)
H2= partial(Hill,v = v2)
parameters = [1.8, 0.67, 0.275, 0.45, 2.95, 0.8]
retLU = []
retProportion = []
for position in range(len(parameters)):
    upper, lower = None, None
    count = 1
    proportion = 0.02
    while True:
        params = parameters.copy()
        params[position] += count*proportion*parameters[position]
        portrait = getPortrait(params,H1, H2)
        
        if count*proportion >=1:
            upper = count
            break
        
        if not isGoodDesign(portrait):
            upper = (count-1)
            break
            
        count+=1
        
        
    count = 1
    while True:
        params = parameters.copy()
        params[position] -= count*proportion*parameters[position]
        if params[position] <=0:
            lower = (count-1)
            break
            
        portrait = getPortrait(params,H1, H2)
        if not isGoodDesign(portrait):
            lower = (count-1)
            break
        count+=1
        
    print(lower*proportion,upper*proportion,position)
    retProportion.append([1-lower*proportion, 1+upper*proportion])
    retLU.append([(1-lower*proportion)*parameters[position], (1+upper*proportion)*parameters[position]])
    print(retLU)
    print(retProportion)

/Users/lunzhang/anaconda3/envs/tensorflow_env/lib/python3.6/site-packages/scipy/optimize/minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/Users/lunzhang/anaconda3/envs/tensorflow_env/lib/python3.6/site-packages/scipy/optimize/minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


0.1 0.22 0
[[1.62, 2.196]]
[[0.9, 1.22]]
0.98 0.04 1
[[1.62, 2.196], [0.013400000000000013, 0.6968000000000001]]
[[0.9, 1.22], [0.020000000000000018, 1.04]]
0.1 0.04 2
[[1.62, 2.196], [0.013400000000000013, 0.6968000000000001], [0.24750000000000003, 0.28600000000000003]]
[[0.9, 1.22], [0.020000000000000018, 1.04], [0.9, 1.04]]
0.04 0.42 3
[[1.62, 2.196], [0.013400000000000013, 0.6968000000000001], [0.24750000000000003, 0.28600000000000003], [0.432, 0.639]]
[[0.9, 1.22], [0.020000000000000018, 1.04], [0.9, 1.04], [0.96, 1.42]]
0.1 0.04 4
[[1.62, 2.196], [0.013400000000000013, 0.6968000000000001], [0.24750000000000003, 0.28600000000000003], [0.432, 0.639], [2.6550000000000002, 3.0680000000000005]]
[[0.9, 1.22], [0.020000000000000018, 1.04], [0.9, 1.04], [0.96, 1.42], [0.9, 1.04]]
0.98 1.0 5
[[1.62, 2.196], [0.013400000000000013, 0.6968000000000001], [0.24750000000000003, 0.28600000000000003], [0.432, 0.639], [2.6550000000000002, 3.0680000000000005], [0.016000000000000014, 1.6]]
[[0.9, 1.

In [35]:
retLU

[[1.62, 2.196],
 [0.013400000000000013, 0.6968000000000001],
 [0.24750000000000003, 0.28600000000000003],
 [0.432, 0.639],
 [2.6550000000000002, 3.0680000000000005],
 [0.016000000000000014, 1.6]]

In [36]:
retProportion

[[0.9, 1.22],
 [0.020000000000000018, 1.04],
 [0.9, 1.04],
 [0.96, 1.42],
 [0.9, 1.04],
 [0.020000000000000018, 2.0]]